In [31]:
#!pip install snorkel

In [1]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
from tensorflow import keras
from sklearn import preprocessing
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import callbacks
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter, RandomVoter, LabelModel

from snorkel.labeling import labeling_function

ABSTAIN = -1

from snorkel.labeling import labeling_function
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc


In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [3]:
print(tf. __version__, np. __version__)

2.1.0 1.17.0


In [192]:
DATA_FOLDER = "../data/clinic/data_small/"
os.listdir(DATA_FOLDER)

['test_with_use_emb.zip',
 'oos_train_with_use_emb.csv',
 'test_with_use_emb.csv',
 '.DS_Store',
 'oos_test_with_use_emb_not_large.csv',
 'oos_train_with_use_emb.zip',
 'val_with_use_emb.csv',
 'val.csv',
 'oos_test.csv',
 'test.csv',
 'train_with_use_emb_not_large.csv',
 'oos_val_with_use_emb_not_large.csv',
 'val_with_use_emb.zip',
 'train_with_use_emb.zip',
 'oos_test_with_use_emb.zip',
 'oos_train.csv',
 'train_with_use_emb.csv',
 'oos_test_with_use_emb.csv',
 'val_with_use_emb_not_large.csv',
 'oos_val.csv',
 'test_with_use_emb_not_large.csv',
 'train.csv',
 'oos_train_with_use_emb_not_large.csv',
 'oos_val_with_use_emb.zip',
 'oos_val_with_use_emb.csv']

In [193]:
df_train = pd.read_csv(DATA_FOLDER + 'train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7500 non-null   object
 1   intent  7500 non-null   object
dtypes: object(2)
memory usage: 117.3+ KB


In [194]:
df_valid = pd.read_csv(DATA_FOLDER + 'val.csv')
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3000 non-null   object
 1   intent  3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [195]:
df_test = pd.read_csv(DATA_FOLDER + 'test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4500 non-null   object
 1   intent  4500 non-null   object
dtypes: object(2)
memory usage: 70.4+ KB


In [150]:
labels = {k:i for i,k in enumerate(set(df_train['intent'].tolist()))}
labels

{'calories': 0,
 'order': 1,
 'travel_notification': 2,
 'make_call': 3,
 'redeem_rewards': 4,
 'credit_limit_change': 5,
 'timer': 6,
 'flip_coin': 7,
 'transactions': 8,
 'what_can_i_ask_you': 9,
 'what_are_your_hobbies': 10,
 'pto_request': 11,
 'credit_limit': 12,
 'rewards_balance': 13,
 'account_blocked': 14,
 'insurance_change': 15,
 'reminder': 16,
 'timezone': 17,
 'how_busy': 18,
 'min_payment': 19,
 'change_volume': 20,
 'tell_joke': 21,
 'pin_change': 22,
 'balance': 23,
 'calendar_update': 24,
 'meeting_schedule': 25,
 'share_location': 26,
 'reset_settings': 27,
 'interest_rate': 28,
 'update_playlist': 29,
 'how_old_are_you': 30,
 'weather': 31,
 'pto_request_status': 32,
 'bill_due': 33,
 'nutrition_info': 34,
 'roll_dice': 35,
 'report_lost_card': 36,
 'next_song': 37,
 'gas': 38,
 'pto_used': 39,
 'repeat': 40,
 'travel_suggestion': 41,
 'jump_start': 42,
 'thank_you': 43,
 'expiration_date': 44,
 'taxes': 45,
 'oil_change_how': 46,
 'credit_score': 47,
 'oil_change_w

In [151]:
labels['oos']= 150

In [152]:
print(labels)

{'calories': 0, 'order': 1, 'travel_notification': 2, 'make_call': 3, 'redeem_rewards': 4, 'credit_limit_change': 5, 'timer': 6, 'flip_coin': 7, 'transactions': 8, 'what_can_i_ask_you': 9, 'what_are_your_hobbies': 10, 'pto_request': 11, 'credit_limit': 12, 'rewards_balance': 13, 'account_blocked': 14, 'insurance_change': 15, 'reminder': 16, 'timezone': 17, 'how_busy': 18, 'min_payment': 19, 'change_volume': 20, 'tell_joke': 21, 'pin_change': 22, 'balance': 23, 'calendar_update': 24, 'meeting_schedule': 25, 'share_location': 26, 'reset_settings': 27, 'interest_rate': 28, 'update_playlist': 29, 'how_old_are_you': 30, 'weather': 31, 'pto_request_status': 32, 'bill_due': 33, 'nutrition_info': 34, 'roll_dice': 35, 'report_lost_card': 36, 'next_song': 37, 'gas': 38, 'pto_used': 39, 'repeat': 40, 'travel_suggestion': 41, 'jump_start': 42, 'thank_you': 43, 'expiration_date': 44, 'taxes': 45, 'oil_change_how': 46, 'credit_score': 47, 'oil_change_when': 48, 'income': 49, 'order_status': 50, 'use

In [153]:
label_list = [k.replace('_',' ') for k in labels.keys()]

In [154]:
label_list[-1] ='out of scope'

In [155]:
print(label_list)

['calories', 'order', 'travel notification', 'make call', 'redeem rewards', 'credit limit change', 'timer', 'flip coin', 'transactions', 'what can i ask you', 'what are your hobbies', 'pto request', 'credit limit', 'rewards balance', 'account blocked', 'insurance change', 'reminder', 'timezone', 'how busy', 'min payment', 'change volume', 'tell joke', 'pin change', 'balance', 'calendar update', 'meeting schedule', 'share location', 'reset settings', 'interest rate', 'update playlist', 'how old are you', 'weather', 'pto request status', 'bill due', 'nutrition info', 'roll dice', 'report lost card', 'next song', 'gas', 'pto used', 'repeat', 'travel suggestion', 'jump start', 'thank you', 'expiration date', 'taxes', 'oil change how', 'credit score', 'oil change when', 'income', 'order status', 'user name', 'cancel reservation', 'change accent', 'international fees', 'definition', 'whisper mode', 'spending history', 'time', 'tire pressure', 'payday', 'reminder update', 'book hotel', 'find 

In [196]:
df_train['label_number'] = df_train['intent'].apply(lambda x: labels.get(x,-1))
df_valid['label_number'] = df_valid['intent'].apply(lambda x: labels.get(x,-1))
df_test['label_number'] = df_test['intent'].apply(lambda x: labels.get(x,-1))

In [157]:
len(df_train),  len(df_test), len(df_valid)

(7500, 4500, 3000)

# Load ZeroShot

In [168]:
from glob import glob
DATA_FOLDER = "../data/clinic/"
all_raw_files = [file for file in glob(DATA_FOLDER+'*/*.csv') if file.find('_with_use_emb')<0]
all_raw_files

['../data/clinic/data_imbalanced/val.csv',
 '../data/clinic/data_imbalanced/oos_test.csv',
 '../data/clinic/data_imbalanced/test.csv',
 '../data/clinic/data_imbalanced/oos_train.csv',
 '../data/clinic/data_imbalanced/oos_val.csv',
 '../data/clinic/data_imbalanced/train.csv',
 '../data/clinic/data_small/val.csv',
 '../data/clinic/data_small/oos_test.csv',
 '../data/clinic/data_small/test.csv',
 '../data/clinic/data_small/oos_train.csv',
 '../data/clinic/data_small/oos_val.csv',
 '../data/clinic/data_small/train.csv',
 '../data/clinic/data_full/val.csv',
 '../data/clinic/data_full/oos_test.csv',
 '../data/clinic/data_full/test.csv',
 '../data/clinic/data_full/oos_train.csv',
 '../data/clinic/data_full/oos_val.csv',
 '../data/clinic/data_full/train.csv',
 '../data/clinic/data_oos_plus/val.csv',
 '../data/clinic/data_oos_plus/oos_test.csv',
 '../data/clinic/data_oos_plus/test.csv',
 '../data/clinic/data_oos_plus/oos_train.csv',
 '../data/clinic/data_oos_plus/oos_val.csv',
 '../data/clinic/

In [39]:
all_text = []
for file_name in all_raw_files:
    dt = pd.read_csv(file_name)[['text']]
    all_text.extend(dt['text'].tolist())

In [40]:
len(all_text), len(set(all_text))

(82975, 23846)

In [31]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [18]:
dt = pd.DataFrame(list(set(all_text)), columns=['text'])

In [42]:
print(set(all_text))

{'lemme know when you were born', 'can you share my credit score with me', 'skip this song and play the next one', 'how many items are on my todo list', "it takes how long to get to mcdonald's in pomona", 'whats euros kroner in 25', 'ai stop talking', 'give me the meaning of alternative', "tell me what's on my todo list", 'does american express charge transaction fees for iceland', 'please let me into my bank account', 'ai can you help me find my phone', 'i would like to apply for a bank of america travel rewards credit card', 'what is the company that made you', 'can you assist me on my fradulent activity on my chase card', 'will you summarize my health benefits', "let's change your accent", 'what is a good beef recipe', 'cash in my credit card points', 'you can call me brandon', 'when will i receive my w2', 'i would like to request that more checkbooks be mailed to me', 'can you tell me when my credit card application will be processed', 'give me your name so i can use you', 'how muc

In [30]:
for text in set(all_text):
    print(text)

lemme know when you were born
how much is my maximum to spend on credit cards
can you slow down your rate of speech
tell me how many a's are in magical
can you share my credit score with me
skip this song and play the next one
hello, how are you
can i speak to you in german
why is my account blocked
how much air is in my tires
it takes how long to get to mcdonald's in pomona
what are popular mountains to hike in arizona
ai stop talking
whats euros kroner in 25
i'm going to india do i need some shots
give me the meaning of alternative
how is my ai doing
i wonder how many points i have earned with my bank of america card
tell me my reminder list
tell me what's on my todo list
what's the average cooking time for cooking wings
does american express charge transaction fees for iceland
my bill shows a purchase from payless shoes but i didn't buy anything
what's the preparation time for spaghetti
please let me into my bank account
my stolen card needs to be reported
i would like to apply for 

i need you to reserve a table for 6:00 pm for three at devon's seafood
where in syracuse can i book a car
set reminder pay taxes on monday
tell me the pin number to my card
define monetary for me please
should i go to okinawa, does it require an international visa
find a recipe for hamburgers
how long does it take milk to expire
which timezone is denver located
does my mastercard have any rewards points remaining
when should i go in and have my tires changed
give me my credit card expiration date
when was i paid most recently
i need la's time zone
did i spend a lot on shoes recently
how many chores are on my todo list
what things are on my shopping list today
you did a good job thank you
let me know the amount of vacation days i have used
can you put detergent on my shopping list cause i'm all out
whats the due date for z bill
please talk quietly
what's the method for requesting a vacation
can you find me a recipe for honey ham
what's the main export by the peoples of greenland
when do

increase your voice box
what's the traffic like on the way to walmart
yes, that is a fact
how much do i need to pay on my bills
i need to up by noon
what would cause my bank account to be frozen
what amount of money do i earn at work
whisper mode, please
when will my package arrive
is the traffic typically bad at noon on the route to the hopsital
how do i do a jump start on my car
any headlines from my area
can you tell me how you're doing today
show me transactions related to travel
address me as "bailey
what are the best ways to build my credit score
let me know when my credit card expire
look up articles on car washing at home
can you find me a salsa recipe
dial steven
what types of questions do you have a response for
i'd like to hear my workout playlist
possibly
tell me if the new york restaurant takes reservations
do i need a travel visa to visit canada
i need a blackberry pie recipe
please freeze my bank account
do you have hobbies
create a reminder for me to send flowers to my 

i'd love to hear when you were born
what is the reason for frozen status on my bank account
why was my card not working at target
what universities are nearby to me and highly rated
i want to hear a joke about elephants
inform my bank that i'll be going to europe
are the ratings for yardhouse good
what timezone is london in
what decent restaurants around here are italian
i need to pay my water bill
when did i take to car to the auto shop
tell me where my phone is
find out if there's a new credit card that suits me
i wish to cancel my reservation for fred at red robin
how many canadian dollars can i exchange for 200 yen
put the dishes on my list of things to do
hey, what's up
are there alot of cars on the road before the stadium
please track my order
where is your home planet
what ingredients do i need for shrimp scampi
what is our purpose
how healthy is blueberrys
when did the oil get changed
how is the time
what ingredients do i need to make spaghetti
do you remember where i left my c

can i used the same type of plug in z as i do in the us
do i have anything on my calendar for may 3
get me a table for five at itta bena at three
i replaced my tires yesterday, when should i get new ones
why did a bunch of christians go off murdering muslims in the middle ages
what is the process of setting up direct deposit
i would like the volume lower
i must say goodbye
is my todo list free
what is on the shopping list
why is there a block on my account
i'm unsure of the proper answer
has there been any changes in the status of my credit card application
do you have the date
can you locate my w-2
go to the next song and play it
throw mopping onto my to do list
i need a flight for under 300 from pittsburgh to atlanta
what are some of the most visited countries for people looking to travel abroad
is chocolate bad for you or good for you
what do i have left in the gas tank
can you assist me in moving money from one account to another
what was said
show allison how to get to me
how long

what are some things that you enjoy
are your real or artificial
an unauthorized transactions has been made on my gap card and i need to report it
call home
how safe is it to travel to mongolia (how safe is mongolia)
how much is the car bill this month
come again with that
double check my reservation under adam for outback steakhouse
how much is available in my savings account
how many japanese yen are in a us dollar
are you a real human being
i want you to reset to brand new please
farewell
can i hear my smooth jazz playlist
is ice cream currently on my shopping list
how do i switch insurance plans
i think my account is blocked but i do not know the reason
thank you!
please repeat what you said
did i receive my paycheck this week
how long does it take to get to el paso on a bus
when did i last have my vehicle checked on
what's the definition of nuclear engineering
do i need a socket converter if i'm going to be in china
how do they say goodbye in hawaii
i want a step by step explanatio

when do i need to pay my annual payment for auto insurance
i need to know the amount of my income
what date did i receive my last paycheck
convert 2 cups to liters
how many days before milk expire
can you tell me why is my bank account frozen
alert my bank i'm checking out seattle for the next month
when did you originate
what is the shipping time for my card
what is the tracking on my package
do you know the name of this song
please find my phone
i need to know when my water bill is due
how much time to drive to the closest bank
what's the amount of air in my tires right now
what is 562 times 400
what's a good place to travel to
i want to update my pin number on my bank of america account
could you inform me of when i last got my car checked
can you tell me my income tax
please change your language setting to spanish now
is there a barbecue restaurant around here
i want an evaluation of my car's tire pressure
could i use olive oil instead of grape seed oil
i'm traveling to ca soon and

is there a recipe for chili i can use
please use your female voice
how long should this be in the oven
at what time will my flight land
i would like to pair this with my new phone
did you grow up here
will we have to make reservations for roca
how long do i need to cook tuna casserole for
which shots do i need before i go to thailand
is flight dl123 due to land as schedule
can i call you adam instead of the current name
when will my car need new oil
if i go to tgifridays at eight, will they be crowded
will i get my time off approved
can you please turn the volume to 4
please set the alarm
what site publishes the most fake news
could you tell me how to direct deposit my check from work into nations bank
confirm my reservation for joe at hoss's
can you tell me what i can ask the ai
is it okay to use applesauce instead of oil
exactly what does altruism mean
how do i go about getting direct deposit in my usaa account
create for me a pto request for march 16-21
how long does it take for dou

tell me the calorie content of frosted flakes
can you find me a brisket recipe
schedule an appointment with the credit counselor for tuesday
how do you unclog a toilet
give my current location to robyn now
if i replaced my tires on christmas eve, when will i need to replace them again
what is 87 divided by 4
what have i spent things on
please create a new alarm for me
for under 700 i need a flight from denver international to new york international on the 3rd
blackberry pie instructions
whats the traffic for the drive to the nearest restaurant
thank you for your assistance
i think all my bills are paid, but can you double check
what kind of indian would you recommend
what's the difference between usd and cad
help me get some new checks since i ran out of them
can you confirm that i have a reservation for february 21 at 6 o'clock
why was my card declined yesterday
what's the month, year, and day that my card expires
i want volume set to 4
what did i do with my phone
hop to the next one


how do i jump my car
i need eggs; add them to my shopping list
how can i go about finding my location right now
how much do i have to pay on taxes and cable
what's the interest rate i have from illinois national
can you raise your speaking speed
please cancel my reservation at red lobster for 6
take $40 from account a and transfer it to account b
how are the burgers at carl jr's
are my tires requiring a fill-up
how much do i have to pay this month
let's use my credit card points
don't be connected to my phone anymore
what day do i get paid next
change color contrast
what is on my schedule between 10:00 am and 11:00 am
this process needs to stop
what kind of insurance do i have
are you a robot
that's correct
create an alarm for 6am
explain to me how transferring my 401k works
a volume of 4 is ideal please
mark down cleaning the bathroom on my list of things to do
i want to hear from you in german
how do i direct deposit my paycheck
can you speak more slowly
can you look to see whether f

In [19]:
scores=[]

In [21]:
dt['text'][0:1]

0                        lemme know when you were born
1                can you share my credit score with me
2                 skip this song and play the next one
3                   how many items are on my todo list
4     it takes how long to get to mcdonald's in pomona
                            ...                       
95       what's the facts about nutrients in rice milk
96                   stop the connection with my phone
97    search for the nearest expressway to the airport
98           am i scheduled to meet with heather today
99                                     what is playing
Name: text, Length: 100, dtype: object

In [184]:
list(np.concatenate(list(zip(['l'+str(i+1) for i in range(10)],['v'+str(i+1) for i in range(10)]))))

['l1',
 'v1',
 'l2',
 'v2',
 'l3',
 'v3',
 'l4',
 'v4',
 'l5',
 'v5',
 'l6',
 'v6',
 'l7',
 'v7',
 'l8',
 'v8',
 'l9',
 'v9',
 'l10',
 'v10']

In [185]:
zeroshot = pd.read_csv(DATA_FOLDER+'00000_03000.csv', header=None,index_col= False, encoding='utf-8',
                       names =['text'] + list(np.concatenate(list(zip(['l'+str(i+1) for i in range(10)],['v'+str(i+1) for i in range(10)])))), 
                       error_bad_lines=False )

In [186]:
zeroshot.head()

,text,l1,v1,l2,v2,l3,v3,l4,v4,l5,...,l6,v6,l7,v7,l8,v8,l9,v9,l10,v10
0,lemme know when you were born,115.0,0.139,26.0,0.059,32.0,0.056,11.0,0.054,30.0,...,3.0,0.036,9.0,0.022,69.0,0.020,97.0,0.018,90.0,0.018
1,can you share my credit score with me,32.0,0.178,11.0,0.148,47.0,0.137,9.0,0.067,69.0,...,3.0,0.027,130.0,0.025,90.0,0.019,50.0,0.016,114.0,0.016
2,skip this song and play the next one,121.0,0.149,87.0,0.076,37.0,0.073,50.0,0.069,27.0,...,71.0,0.033,1.0,0.032,29.0,0.030,147.0,0.029,40.0,0.025
3,how many items are on my todo list,126.0,0.435,18.0,0.115,50.0,0.055,1.0,0.026,16.0,...,86.0,0.016,9.0,0.016,147.0,0.015,79.0,0.014,39.0,0.013
4,it takes how long to get to mcdonald's in pomona,148.0,0.151,41.0,0.092,58.0,0.082,82.0,0.053,75.0,...,121.0,0.039,112.0,0.034,6.0,0.030,18.0,0.029,2.0,0.027


In [189]:
zeroshot.dropna(inplace=True)

In [190]:
for i in range(10):
    zeroshot['l'+str(i+1)] = zeroshot['l'+str(i+1)].astype('int')

In [45]:
zeroshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23801 entries, 0 to 23800
Columns: 152 entries, 0 to 151
dtypes: float64(151), object(1)
memory usage: 27.6+ MB


In [191]:
zeroshot.head()

,text,l1,v1,l2,v2,l3,v3,l4,v4,l5,...,l6,v6,l7,v7,l8,v8,l9,v9,l10,v10
0,lemme know when you were born,115,0.139,26,0.059,32,0.056,11,0.054,30,...,3,0.036,9,0.022,69,0.020,97,0.018,90,0.018
1,can you share my credit score with me,32,0.178,11,0.148,47,0.137,9,0.067,69,...,3,0.027,130,0.025,90,0.019,50,0.016,114,0.016
2,skip this song and play the next one,121,0.149,87,0.076,37,0.073,50,0.069,27,...,71,0.033,1,0.032,29,0.030,147,0.029,40,0.025
3,how many items are on my todo list,126,0.435,18,0.115,50,0.055,1,0.026,16,...,86,0.016,9,0.016,147,0.015,79,0.014,39,0.013
4,it takes how long to get to mcdonald's in pomona,148,0.151,41,0.092,58,0.082,82,0.053,75,...,121,0.039,112,0.034,6,0.030,18,0.029,2,0.027


In [197]:
df_train = pd.merge(df_train, zeroshot, how= 'left', on='text')

In [198]:
df_valid = pd.merge(df_valid, zeroshot, how= 'left', on='text')
df_test = pd.merge(df_test, zeroshot, how= 'left', on='text')

In [202]:
df_train.head(10)

,text,intent,label_number,l1,v1,l2,v2,l3,v3,l4,...,l6,v6,l7,v7,l8,v8,l9,v9,l10,v10
5,skip and play next song,next_song,37,121.0,0.105,27.0,0.081,37.0,0.062,87.0,...,137.0,0.047,29.0,0.036,71.0,0.035,91.0,0.027,40.0,0.020
55,how much did i spend on gas last week,spending_history,57,38.0,0.306,57.0,0.054,9.0,0.043,8.0,...,94.0,0.033,99.0,0.029,114.0,0.022,85.0,0.021,11.0,0.015
72,call an uber that fits 5 people that are on th...,uber,116,41.0,0.133,116.0,0.106,3.0,0.100,112.0,...,121.0,0.036,82.0,0.028,1.0,0.028,50.0,0.026,130.0,0.023
78,what are the estimates for the car's highway mpg,mpg,94,94.0,0.096,2.0,0.071,41.0,0.065,32.0,...,38.0,0.041,3.0,0.041,132.0,0.031,9.0,0.028,99.0,0.025
91,tell me why was my card declined at the zoo,card_declined,71,11.0,0.121,32.0,0.118,9.0,0.096,71.0,...,3.0,0.043,39.0,0.020,57.0,0.019,92.0,0.019,87.0,0.018
95,how many days until my next day off,next_holiday,120,115.0,0.112,58.0,0.076,24.0,0.069,78.0,...,6.0,0.047,1.0,0.044,32.0,0.032,11.0,0.023,120.0,0.021
99,how long can you go between oil changes,oil_change_when,48,48.0,0.114,46.0,0.102,9.0,0.049,11.0,...,125.0,0.032,147.0,0.026,58.0,0.024,96.0,0.021,94.0,0.020
102,is my tire pressure okay,tire_pressure,59,59.0,0.385,32.0,0.055,50.0,0.042,103.0,...,130.0,0.028,100.0,0.026,9.0,0.022,97.0,0.021,69.0,0.015
123,i didn't hear what you said,repeat,40,147.0,0.068,56.0,0.060,97.0,0.045,69.0,...,71.0,0.027,75.0,0.022,107.0,0.022,90.0,0.022,39.0,0.016
132,do i need any tools to change the oil in my car,oil_change_how,46,11.0,0.122,48.0,0.121,32.0,0.087,9.0,...,147.0,0.037,50.0,0.033,90.0,0.026,3.0,0.025,1.0,0.025


In [203]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)
df_valid.dropna(inplace=True)

In [71]:
zcolumns = [str(i) for i in range(151)]
zcolumns

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [204]:
df_train.columns

Index(['text', 'intent', 'label_number', 'l1', 'v1', 'l2', 'v2', 'l3', 'v3',
       'l4', 'v4', 'l5', 'v5', 'l6', 'v6', 'l7', 'v7', 'l8', 'v8', 'l9', 'v9',
       'l10', 'v10'],
      dtype='object')

In [205]:
threshold1 = round(10* 1/151, 3)
threshold1

0.066

In [225]:
threshold2 = round(7* 1/151, 3)
threshold2

0.046

In [226]:
threshold3 = round(5* 1/151, 3)
threshold3

0.033

In [ ]:
threshold1 = 0.

In [208]:
@labeling_function()
def r0(x):  
    if x.v1>=threshold1:
        return int(x.l1) 
    return -1
    
@labeling_function()
def r1(x):
    if x.v2>=threshold1 and x.v1- x.v2<0.1:
        return int(x.l2)     
    return -1

@labeling_function()
def r2(x):
    if x.v3>=threshold1 and x.v2- x.v3<0.1:
        return int(x.l3)          
    return -1

In [227]:
@labeling_function()
def r0(x):  
    if x.v1>=threshold1:
        return int(x.l1) 
    return -1
    
@labeling_function()
def r1(x):
    if x.v2>=threshold2 and x.v1- x.v2<0.1:
        return int(x.l2)     
    return -1

@labeling_function()
def r2(x):
    if x.v3>=threshold3 and x.v2- x.v3<0.1:
        return int(x.l3)          
    return -1

In [228]:
lfs = [r0,r1,r2]#, r3, r4, r5, r6, r7, r8,r9,r10, r11, r12, r13, r14, r15, r16, r17, r18]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 700/700 [00:00<00:00, 10802.98it/s]


In [229]:
coverages = (L_train != ABSTAIN).mean(axis=0)
print("Coverages:")
for  cov, lf in zip(coverages, lfs):
    print("  {}: {:.2f}%".format(lf.name, 100*cov))

Coverages:
  r0: 96.86%
  r1: 64.43%
  r2: 91.86%


In [230]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
r0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0.968571,0.911429,0.911429
r1,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16...",0.644286,0.644286,0.644286
r2,2,"[1, 2, 3, 4, 6, 8, 9, 11, 12, 13, 14, 16, 17, ...",0.918571,0.907143,0.907143


In [231]:
random_voter = RandomVoter(cardinality=151)
preds_train = random_voter.predict(L=L_train)

In [232]:
majority_model = MajorityLabelVoter(cardinality=151)
preds_train = majority_model.predict(L=L_train)

In [233]:
label_model = LabelModel(cardinality=151, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:04<00:00, 111.35epoch/s]
INFO:root:Finished Training


In [234]:
L_test = applier.apply(df=df_test)

100%|██████████| 369/369 [00:00<00:00, 11171.89it/s]


In [235]:
random_model_acc = random_voter.score(L=L_test, Y=df_test['label_number'], tie_break_policy="random")["accuracy"]
print(f"{'Random Model Accuracy:':<25} {random_model_acc * 100:.1f}%")

majority_acc = majority_model.score(L=L_test, Y=df_test['label_number'], tie_break_policy="random")["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test['label_number'], tie_break_policy="random")["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Random Model Accuracy:    1.4%
Majority Vote Accuracy:   33.1%
Label Model Accuracy:     42.8%
